In [1]:
import urllib
import requests
import regex as re
import xlsxwriter
from bs4 import BeautifulSoup

In [2]:
# Get bird names from pre-existing text file
bird_data_file = open("ebird_istanbul.txt", "r")
bird_data = bird_data_file.read()
bird_names = bird_data.split("\n")

In [3]:
# Open worksheet and initialize column names
workbook = xlsxwriter.Workbook('bird_wiki_descriptions.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write('A1', 'bird_name')
worksheet.write('B1', 'wiki_desc')

0

In [4]:
special_cases = ["Delicate Prinia", "Ruff", "Desert Finch", "Mallard", "Turkestan Short-toed Lark", "Short-toed Snake-Eagle",
                 "Gray-headed Swamphen", "Yelkouan Shearwater", "Great Gray Shrike", "White-tailed Eagle", "Mediterranean Short-toed Lark",
                "Greater Scaup", "Tundra Swan", "Sociable Lapwing", "Gray-necked Bunting", "Arabian Lark", "Gray-headed Lapwing",
                "Brant"]

special_cases_count = ["desert finch", "mallard", "yelkouan shearwater", "White-tailed Eagle","Greater Scaup", "tundra swan"
                      "sociable lapwing", "gray-headed lapwing"]

In [ ]:
def get_wiki_description(title):
    page_url =  'https://en.wikipedia.org/wiki/' + title
    # Query the website and return the html to the variable 'page'
    page = requests.get(page_url).text
    # Parse the html using beautiful soup and store in variable 'soup'
    soup = BeautifulSoup(page, 'html.parser')
    for p in soup.find_all('p'):
        #print("BEGINNING")
        print(p)
        if len(p.get_text()) > 100:
            return p.get_text()

In [ ]:
# Out of 494 entries 18 were faulty. Since pages for those entries are 
# quite irregular we are going to extract them with different methods
def get_wiki_special_description(title):
    title = title.lower()
    # Ones we can obtain with <p>'s specific position
    if title in special_cases_count:
        return get_desc_with_count(title, 3)
    if title == "Ruff":
        title += " (bird)"
    get_wiki_description(title)

In [ ]:
# Get the 'count'th <p> 
def get_desc_with_count(title, count):
    page_url =  'https://en.wikipedia.org/wiki/' + title
    # Query the website and return the html to the variable 'page'
    page = requests.get(page_url).text
    # Parse the html using beautiful soup and store in variable 'soup'
    soup = BeautifulSoup(page, 'html.parser')
    counter = 0
    for p in soup.find_all('p'):
        counter += 1
        if counter == 3:
            return p.get_text()
    return

In [ ]:
row_num = 2
for bird_name in bird_names:
    # Stopping before main wikipedia page also gets scraped
    if row_num == len(bird_names) + 2:
        break
    worksheet.write("A" + str(row_num), bird_name)
    if bird_name in special_cases:
        worksheet.write("B" + str(row_num), get_wiki_special_description(bird_name))
    else:
        worksheet.write("B" + str(row_num), get_wiki_description(bird_name))
    row_num += 1

In [ ]:
workbook.close()